In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.under_sampling import NearMiss
nm= NearMiss()

X_nm, y_nm= nm.fit_resample(X, y)
print(f'Near Miss: {Counter(y_nm)}')

Near Miss: Counter({0: 4561, 1: 4561})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_nm_train, y_nm_train= nm.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_nm= LogisticRegression(random_state=15)
lr_nm.fit(X_nm_train, y_nm_train)
y_pred_lr_nm= lr_nm.predict(X_test)
y_pred_prob_lr_nm= lr_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_nm, labels= lr_nm.classes_)
print(classification_report(y_test, y_pred_lr_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_nm))

              precision    recall  f1-score   support

           0       0.95      0.32      0.48      8517
           1       0.12      0.85      0.21       912

    accuracy                           0.37      9429
   macro avg       0.53      0.58      0.34      9429
weighted avg       0.87      0.37      0.45      9429

Precision: 0.11753048780487804
Accuracy: 0.37108919291547354
AUC: 0.6556981496243838


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_nm= DecisionTreeClassifier(random_state=15)
dt_nm.fit(X_nm_train, y_nm_train)
y_pred_dt_nm= dt_nm.predict(X_test)
y_pred_prob_dt_nm= dt_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_nm, labels= dt_nm.classes_)
print(classification_report(y_test, y_pred_dt_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_nm))

              precision    recall  f1-score   support

           0       0.95      0.36      0.52      8517
           1       0.12      0.83      0.21       912

    accuracy                           0.41      9429
   macro avg       0.54      0.59      0.37      9429
weighted avg       0.87      0.41      0.49      9429

Precision: 0.12193152454780362
Accuracy: 0.4067239367907519
AUC: 0.5951959599892064


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_nm= GridSearchCV(estimator= dt_nm, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_nm.fit(X_nm_train, y_nm_train)

print("Best Parameters:", gs_dt_nm.best_params_)
print("Best Precision Score:", gs_dt_nm.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.9175216966349927


In [12]:
tuned_dt_nm= gs_dt_nm.best_estimator_
y_pred_tuned_dt_nm= tuned_dt_nm.predict(X_test)
y_pred_prob_tuned_dt_nm= tuned_dt_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_nm, labels= tuned_dt_nm.classes_)
print(classification_report(y_test, y_pred_tuned_dt_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_nm))

              precision    recall  f1-score   support

           0       0.96      0.53      0.68      8517
           1       0.15      0.78      0.25       912

    accuracy                           0.56      9429
   macro avg       0.56      0.66      0.47      9429
weighted avg       0.88      0.56      0.64      9429

Precision: 0.15196599362380447
Accuracy: 0.5559444267684802
AUC: 0.6866080146209129


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_nm= KNeighborsClassifier()
knn_nm.fit(X_nm_train, y_nm_train)
y_pred_knn_nm= knn_nm.predict(X_test)
y_pred_prob_knn_nm= knn_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_nm, labels= knn_nm.classes_)
print(classification_report(y_test, y_pred_knn_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_nm))

              precision    recall  f1-score   support

           0       0.94      0.44      0.60      8517
           1       0.13      0.74      0.21       912

    accuracy                           0.47      9429
   macro avg       0.53      0.59      0.41      9429
weighted avg       0.86      0.47      0.57      9429

Precision: 0.1256011838697743
Accuracy: 0.4739633046982713
AUC: 0.6397091652608097


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_nm= GridSearchCV(estimator= knn_nm, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_nm.fit(X_nm_train, y_nm_train)
print("Best Parameters:", gs_knn_nm.best_params_)
print("Best Precision Score:", gs_knn_nm.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 38, 'weights': 'uniform'}
Best Precision Score: 0.9679131315975195


In [15]:
tuned_knn_nm= gs_knn_nm.best_estimator_
y_pred_tuned_knn_nm= tuned_knn_nm.predict(X_test)
y_pred_prob_tuned_knn_nm= tuned_knn_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_nm, labels= tuned_knn_nm.classes_)
print(classification_report(y_test, y_pred_tuned_knn_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_nm))

              precision    recall  f1-score   support

           0       0.95      0.71      0.81      8517
           1       0.19      0.63      0.29       912

    accuracy                           0.70      9429
   macro avg       0.57      0.67      0.55      9429
weighted avg       0.87      0.70      0.76      9429

Precision: 0.18902038132807364
Accuracy: 0.7026195778979744
AUC: 0.7368392085797446


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_nm= GaussianNB()
nb_nm.fit(X_nm_train, y_nm_train)
y_pred_nb_nm= nb_nm.predict(X_test)
y_pred_prob_nb_nm= nb_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_nm, labels= nb_nm.classes_)
print(classification_report(y_test, y_pred_nb_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_nm))

              precision    recall  f1-score   support

           0       0.92      0.49      0.64      8517
           1       0.11      0.61      0.19       912

    accuracy                           0.50      9429
   macro avg       0.52      0.55      0.42      9429
weighted avg       0.84      0.50      0.60      9429

Precision: 0.11357113571135712
Accuracy: 0.5034468130236505
AUC: 0.5684936242066949


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_nm= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_nm.fit(X_nm_train, y_nm_train)
y_pred_svc_nm= svc_nm.predict(X_test)
y_pred_prob_svc_nm= svc_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_nm, labels= svc_nm.classes_)
print(classification_report(y_test, y_pred_svc_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_nm))

              precision    recall  f1-score   support

           0       0.93      0.09      0.16      8517
           1       0.10      0.94      0.18       912

    accuracy                           0.17      9429
   macro avg       0.51      0.51      0.17      9429
weighted avg       0.85      0.17      0.16      9429

Precision: 0.0990282276723739
Accuracy: 0.16809841976879838
AUC: 0.5895879809009432


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_nm= RandomForestClassifier(random_state=15)
rf_nm.fit(X_nm_train, y_nm_train)
y_pred_rf_nm= rf_nm.predict(X_test)
y_pred_prob_rf_nm= rf_nm.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_nm, labels= rf_nm.classes_)
print(classification_report(y_test, y_pred_rf_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_nm))

              precision    recall  f1-score   support

           0       0.96      0.35      0.51      8517
           1       0.12      0.86      0.22       912

    accuracy                           0.40      9429
   macro avg       0.54      0.61      0.36      9429
weighted avg       0.88      0.40      0.48      9429

Precision: 0.12421085858585859
Accuracy: 0.3982394739633047
AUC: 0.7135283741083365


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_nm= GridSearchCV(estimator= rf_nm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_nm.fit(X_nm_train, y_nm_train)
print("Best Parameters:", gs_tuned_rf_nm.best_params_)
print("Best Precision Score:", gs_tuned_rf_nm.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Precision Score: 0.9140823607489402


In [21]:
tuned_rf_nm= gs_tuned_rf_nm.best_estimator_
y_pred_tuned_rf_nm= tuned_rf_nm.predict(X_test)
y_pred_prob_tuned_rf_nm= tuned_rf_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_nm, labels= tuned_rf_nm.classes_)
print(classification_report(y_test, y_pred_tuned_rf_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_nm))

              precision    recall  f1-score   support

           0       0.96      0.43      0.60      8517
           1       0.14      0.84      0.24       912

    accuracy                           0.47      9429
   macro avg       0.55      0.64      0.42      9429
weighted avg       0.88      0.47      0.56      9429

Precision: 0.13761632068718682
Accuracy: 0.4737511931275851
AUC: 0.7291732968531461


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_nm= AdaBoostClassifier(random_state=15)
ada_nm.fit(X_nm_train, y_nm_train)
y_pred_ada_nm= ada_nm.predict(X_test)
y_pred_prob_ada_nm= ada_nm.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_nm, labels= ada_nm.classes_)
print(classification_report(y_test, y_pred_ada_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_nm))

              precision    recall  f1-score   support

           0       0.95      0.32      0.48      8517
           1       0.12      0.84      0.21       912

    accuracy                           0.37      9429
   macro avg       0.53      0.58      0.34      9429
weighted avg       0.87      0.37      0.45      9429

Precision: 0.11707465409761289
Accuracy: 0.3690741329939548
AUC: 0.5777326925097175


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_nm= GridSearchCV(estimator= ada_nm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_nm.fit(X_nm_train, y_nm_train)
print("Best Parameters:", gs_ada_nm.best_params_)
print("Best Precision Score:", gs_ada_nm.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9132621896793861


In [25]:
tuned_ada_nm= gs_ada_nm.best_estimator_
y_pred_tuned_ada_nm= tuned_ada_nm.predict(X_test)
y_pred_prob_tuned_ada_nm= tuned_ada_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_nm, labels= tuned_ada_nm.classes_)
print(classification_report(y_test, y_pred_tuned_ada_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_nm))

              precision    recall  f1-score   support

           0       0.95      0.32      0.48      8517
           1       0.12      0.85      0.21       912

    accuracy                           0.37      9429
   macro avg       0.53      0.58      0.34      9429
weighted avg       0.87      0.37      0.45      9429

Precision: 0.11786259541984732
Accuracy: 0.3723618623395906
AUC: 0.5793541915137732


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_nm= GradientBoostingClassifier(random_state=15)
grb_nm.fit(X_nm_train, y_nm_train)
y_pred_grb_nm= grb_nm.predict(X_test)
y_pred_prob_grb_nm= grb_nm.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_nm, labels= grb_nm.classes_)
print(classification_report(y_test, y_pred_grb_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_nm))

              precision    recall  f1-score   support

           0       0.96      0.40      0.57      8517
           1       0.13      0.86      0.23       912

    accuracy                           0.44      9429
   macro avg       0.55      0.63      0.40      9429
weighted avg       0.88      0.44      0.53      9429

Precision: 0.13308436598200646
Accuracy: 0.4447979637289214
AUC: 0.7161747840747814


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_nm= GridSearchCV(estimator= grb_nm, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_nm.fit(X_nm_train, y_nm_train)

print("Best Parameters:", gs_grb_nm.best_params_)
print("Best Precision Score:", gs_grb_nm.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9110761691931355


In [28]:
tuned_grb_nm= gs_grb_nm.best_estimator_
y_pred_tuned_grb_nm= tuned_grb_nm.predict(X_test)
y_pred_prob_tuned_grb_nm= tuned_grb_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_nm, labels= tuned_grb_nm.classes_)
print(classification_report(y_test, y_pred_tuned_grb_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_nm))

              precision    recall  f1-score   support

           0       0.96      0.40      0.57      8517
           1       0.13      0.86      0.23       912

    accuracy                           0.44      9429
   macro avg       0.55      0.63      0.40      9429
weighted avg       0.88      0.44      0.53      9429

Precision: 0.13308436598200646
Accuracy: 0.4447979637289214
AUC: 0.7161747840747814


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_nm= XGBClassifier(random_state=15)
xgb_nm.fit(X_nm_train, y_nm_train)
y_pred_xgb_nm= xgb_nm.predict(X_test)
y_pred_prob_xgb_nm= xgb_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_nm, labels= xgb_nm.classes_)
print(classification_report(y_test, y_pred_xgb_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_nm))

              precision    recall  f1-score   support

           0       0.96      0.35      0.51      8517
           1       0.12      0.85      0.21       912

    accuracy                           0.40      9429
   macro avg       0.54      0.60      0.36      9429
weighted avg       0.88      0.40      0.48      9429

Precision: 0.12298450837812204
Accuracy: 0.39739102768055995
AUC: 0.6894212091812247


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_nm= GridSearchCV(estimator= xgb_nm, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_nm.fit(X_nm_train, y_nm_train)

print("Best Parameters:", gs_xgb_nm.best_params_)
print("Best Precision Score:", gs_xgb_nm.best_score_)

tuned_xgb_nm= gs_xgb_nm.best_estimator_
y_pred_tuned_xgb_nm= tuned_xgb_nm.predict(X_test)
y_pred_prob_tuned_xgb_nm= tuned_xgb_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_nm, labels= tuned_xgb_nm.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_nm))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9127529570281168
              precision    recall  f1-score   support

           0       0.96      0.37      0.53      8517
           1       0.13      0.85      0.22       912

    accuracy                           0.42      9429
   macro avg       0.54      0.61      0.38      9429
weighted avg       0.88      0.42      0.50      9429

Precision: 0.12646675358539766
Accuracy: 0.4171174037543748
AUC: 0.6991046930905991


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_nm= LGBMClassifier(random_state=15)
lgm_nm.fit(X_nm_train, y_nm_train)
y_pred_lgm_nm= lgm_nm.predict(X_test)
y_pred_prob_lgm_nm= lgm_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_nm, labels= lgm_nm.classes_)
print(classification_report(y_test, y_pred_lgm_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_nm))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.273024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 210
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.96      0.36      0.52      8517
           1       0.12      0.85      0.22       912

    accuracy                           0.41      9429
   macro avg       0.54      0.60      0.37      9429
weighted avg       0.88      0.41      0.49      9429

Precision: 0.12397885631907736
Accuracy: 0.4053452115812918
AUC: 0.6966872498552945


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_nm= GridSearchCV(estimator= lgm_nm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_nm.fit(X_nm_train, y_nm_train)

print("Best Parameters:", gs_lgm_nm.best_params_)
print("Best Precision Score:", gs_lgm_nm.best_score_)

tuned_lgm_nm= gs_lgm_nm.best_estimator_
y_pred_tuned_lgm_nm= tuned_lgm_nm.predict(X_test)
y_pred_prob_tuned_lgm_nm= tuned_lgm_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_nm, labels= tuned_lgm_nm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_nm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008755 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 210
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.8995452669910089
[LightGBM] [Warning] min_data_in_leaf is set=30, min_

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_nm= CatBoostClassifier(random_state=15)
cat_nm.fit(X_nm_train, y_nm_train)
y_pred_cat_nm= cat_nm.predict(X_test)
y_pred_prob_cat_nm= cat_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_nm, labels= cat_nm.classes_)
print(classification_report(y_test, y_pred_cat_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_nm))

Learning rate set to 0.024073
0:	learn: 0.6704175	total: 150ms	remaining: 2m 29s
1:	learn: 0.6509708	total: 157ms	remaining: 1m 18s
2:	learn: 0.6341720	total: 165ms	remaining: 54.8s
3:	learn: 0.6151230	total: 174ms	remaining: 43.4s
4:	learn: 0.5982568	total: 183ms	remaining: 36.4s
5:	learn: 0.5840094	total: 190ms	remaining: 31.5s
6:	learn: 0.5692597	total: 198ms	remaining: 28s
7:	learn: 0.5573880	total: 206ms	remaining: 25.5s
8:	learn: 0.5452546	total: 212ms	remaining: 23.4s
9:	learn: 0.5337309	total: 221ms	remaining: 21.9s
10:	learn: 0.5217206	total: 228ms	remaining: 20.5s
11:	learn: 0.5108233	total: 236ms	remaining: 19.4s
12:	learn: 0.5006577	total: 245ms	remaining: 18.6s
13:	learn: 0.4911242	total: 255ms	remaining: 18s
14:	learn: 0.4824734	total: 262ms	remaining: 17.2s
15:	learn: 0.4748217	total: 278ms	remaining: 17.1s
16:	learn: 0.4670760	total: 290ms	remaining: 16.8s
17:	learn: 0.4593983	total: 297ms	remaining: 16.2s
18:	learn: 0.4524875	total: 306ms	remaining: 15.8s
19:	learn: 0.

164:	learn: 0.2813161	total: 1.44s	remaining: 7.3s
165:	learn: 0.2810285	total: 1.45s	remaining: 7.3s
166:	learn: 0.2808249	total: 1.46s	remaining: 7.28s
167:	learn: 0.2806021	total: 1.47s	remaining: 7.26s
168:	learn: 0.2804522	total: 1.47s	remaining: 7.25s
169:	learn: 0.2802752	total: 1.48s	remaining: 7.23s
170:	learn: 0.2800238	total: 1.49s	remaining: 7.21s
171:	learn: 0.2798527	total: 1.5s	remaining: 7.2s
172:	learn: 0.2796604	total: 1.5s	remaining: 7.18s
173:	learn: 0.2793746	total: 1.51s	remaining: 7.17s
174:	learn: 0.2791733	total: 1.52s	remaining: 7.16s
175:	learn: 0.2790255	total: 1.53s	remaining: 7.15s
176:	learn: 0.2788663	total: 1.53s	remaining: 7.13s
177:	learn: 0.2786871	total: 1.54s	remaining: 7.12s
178:	learn: 0.2784607	total: 1.55s	remaining: 7.1s
179:	learn: 0.2781699	total: 1.56s	remaining: 7.1s
180:	learn: 0.2780149	total: 1.57s	remaining: 7.09s
181:	learn: 0.2778227	total: 1.57s	remaining: 7.07s
182:	learn: 0.2775108	total: 1.58s	remaining: 7.06s
183:	learn: 0.27732

323:	learn: 0.2581135	total: 2.69s	remaining: 5.6s
324:	learn: 0.2579819	total: 2.69s	remaining: 5.59s
325:	learn: 0.2579200	total: 2.7s	remaining: 5.58s
326:	learn: 0.2578611	total: 2.71s	remaining: 5.57s
327:	learn: 0.2577679	total: 2.72s	remaining: 5.57s
328:	learn: 0.2576553	total: 2.72s	remaining: 5.56s
329:	learn: 0.2575193	total: 2.73s	remaining: 5.55s
330:	learn: 0.2574385	total: 2.74s	remaining: 5.54s
331:	learn: 0.2573871	total: 2.75s	remaining: 5.53s
332:	learn: 0.2572930	total: 2.75s	remaining: 5.51s
333:	learn: 0.2572196	total: 2.76s	remaining: 5.5s
334:	learn: 0.2570495	total: 2.77s	remaining: 5.49s
335:	learn: 0.2569789	total: 2.77s	remaining: 5.48s
336:	learn: 0.2569365	total: 2.78s	remaining: 5.47s
337:	learn: 0.2568137	total: 2.79s	remaining: 5.46s
338:	learn: 0.2567842	total: 2.8s	remaining: 5.45s
339:	learn: 0.2566792	total: 2.8s	remaining: 5.44s
340:	learn: 0.2566105	total: 2.81s	remaining: 5.43s
341:	learn: 0.2564029	total: 2.82s	remaining: 5.42s
342:	learn: 0.256

504:	learn: 0.2402656	total: 4.09s	remaining: 4s
505:	learn: 0.2402067	total: 4.09s	remaining: 4s
506:	learn: 0.2401433	total: 4.1s	remaining: 3.99s
507:	learn: 0.2400289	total: 4.11s	remaining: 3.98s
508:	learn: 0.2399858	total: 4.12s	remaining: 3.97s
509:	learn: 0.2399054	total: 4.12s	remaining: 3.96s
510:	learn: 0.2398225	total: 4.13s	remaining: 3.95s
511:	learn: 0.2397515	total: 4.14s	remaining: 3.94s
512:	learn: 0.2396578	total: 4.14s	remaining: 3.93s
513:	learn: 0.2395989	total: 4.15s	remaining: 3.92s
514:	learn: 0.2395424	total: 4.16s	remaining: 3.92s
515:	learn: 0.2394716	total: 4.17s	remaining: 3.91s
516:	learn: 0.2394071	total: 4.17s	remaining: 3.9s
517:	learn: 0.2393559	total: 4.18s	remaining: 3.89s
518:	learn: 0.2392494	total: 4.19s	remaining: 3.88s
519:	learn: 0.2391412	total: 4.2s	remaining: 3.87s
520:	learn: 0.2391042	total: 4.2s	remaining: 3.86s
521:	learn: 0.2390460	total: 4.21s	remaining: 3.85s
522:	learn: 0.2389447	total: 4.21s	remaining: 3.84s
523:	learn: 0.2388464	

666:	learn: 0.2290171	total: 5.34s	remaining: 2.66s
667:	learn: 0.2289764	total: 5.34s	remaining: 2.66s
668:	learn: 0.2289341	total: 5.35s	remaining: 2.65s
669:	learn: 0.2289091	total: 5.36s	remaining: 2.64s
670:	learn: 0.2288691	total: 5.37s	remaining: 2.63s
671:	learn: 0.2287985	total: 5.38s	remaining: 2.62s
672:	learn: 0.2287316	total: 5.38s	remaining: 2.62s
673:	learn: 0.2286677	total: 5.39s	remaining: 2.61s
674:	learn: 0.2286190	total: 5.4s	remaining: 2.6s
675:	learn: 0.2285252	total: 5.4s	remaining: 2.59s
676:	learn: 0.2284418	total: 5.41s	remaining: 2.58s
677:	learn: 0.2283891	total: 5.42s	remaining: 2.57s
678:	learn: 0.2282883	total: 5.43s	remaining: 2.57s
679:	learn: 0.2282524	total: 5.43s	remaining: 2.56s
680:	learn: 0.2282022	total: 5.44s	remaining: 2.55s
681:	learn: 0.2281529	total: 5.45s	remaining: 2.54s
682:	learn: 0.2281169	total: 5.46s	remaining: 2.53s
683:	learn: 0.2280652	total: 5.46s	remaining: 2.52s
684:	learn: 0.2279712	total: 5.47s	remaining: 2.52s
685:	learn: 0.2

827:	learn: 0.2198503	total: 6.59s	remaining: 1.37s
828:	learn: 0.2198108	total: 6.6s	remaining: 1.36s
829:	learn: 0.2197591	total: 6.61s	remaining: 1.35s
830:	learn: 0.2196486	total: 6.62s	remaining: 1.35s
831:	learn: 0.2195975	total: 6.63s	remaining: 1.34s
832:	learn: 0.2195142	total: 6.64s	remaining: 1.33s
833:	learn: 0.2194543	total: 6.66s	remaining: 1.32s
834:	learn: 0.2193952	total: 6.67s	remaining: 1.32s
835:	learn: 0.2193646	total: 6.67s	remaining: 1.31s
836:	learn: 0.2193199	total: 6.68s	remaining: 1.3s
837:	learn: 0.2192409	total: 6.69s	remaining: 1.29s
838:	learn: 0.2191830	total: 6.7s	remaining: 1.28s
839:	learn: 0.2191066	total: 6.7s	remaining: 1.28s
840:	learn: 0.2190072	total: 6.71s	remaining: 1.27s
841:	learn: 0.2189652	total: 6.72s	remaining: 1.26s
842:	learn: 0.2189303	total: 6.73s	remaining: 1.25s
843:	learn: 0.2188999	total: 6.73s	remaining: 1.24s
844:	learn: 0.2188669	total: 6.74s	remaining: 1.24s
845:	learn: 0.2188272	total: 6.75s	remaining: 1.23s
846:	learn: 0.21

987:	learn: 0.2115923	total: 7.85s	remaining: 95.3ms
988:	learn: 0.2115348	total: 7.85s	remaining: 87.4ms
989:	learn: 0.2114746	total: 7.86s	remaining: 79.4ms
990:	learn: 0.2114481	total: 7.87s	remaining: 71.5ms
991:	learn: 0.2113318	total: 7.88s	remaining: 63.5ms
992:	learn: 0.2113001	total: 7.88s	remaining: 55.6ms
993:	learn: 0.2112586	total: 7.89s	remaining: 47.6ms
994:	learn: 0.2111923	total: 7.9s	remaining: 39.7ms
995:	learn: 0.2111432	total: 7.91s	remaining: 31.8ms
996:	learn: 0.2110951	total: 7.92s	remaining: 23.8ms
997:	learn: 0.2110897	total: 7.92s	remaining: 15.9ms
998:	learn: 0.2110433	total: 7.93s	remaining: 7.94ms
999:	learn: 0.2109834	total: 7.94s	remaining: 0us
              precision    recall  f1-score   support

           0       0.96      0.33      0.49      8517
           1       0.12      0.86      0.21       912

    accuracy                           0.38      9429
   macro avg       0.54      0.59      0.35      9429
weighted avg       0.88      0.38      0.46

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_nm= GridSearchCV(estimator= cat_nm, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_nm.fit(X_nm_train, y_nm_train)

print("Best Parameters:", gs_cat_nm.best_params_)
print("Best Precision Score:", gs_cat_nm.best_score_)

tuned_cat_nm= gs_cat_nm.best_estimator_
y_pred_tuned_cat_nm= tuned_cat_nm.predict(X_test)
y_pred_prob_tuned_cat_nm= tuned_cat_nm.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_nm, labels= tuned_cat_nm.classes_)
print(classification_report(y_test, y_pred_tuned_cat_nm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_nm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_nm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_nm))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6359657	total: 7.5ms	remaining: 742ms
1:	learn: 0.5737668	total: 15.1ms	remaining: 738ms
2:	learn: 0.5383405	total: 21.8ms	remaining: 704ms
3:	learn: 0.5069762	total: 30ms	remaining: 721ms
4:	learn: 0.4792788	total: 37.3ms	remaining: 709ms
5:	learn: 0.4582670	total: 45.3ms	remaining: 710ms
6:	learn: 0.4399576	total: 54.4ms	remaining: 723ms
7:	learn: 0.4238040	total: 65.6ms	remaining: 755ms
8:	learn: 0.4116685	total: 76ms	remaining: 768ms
9:	learn: 0.3994650	total: 84.3ms	remaining: 758ms
10:	learn: 0.3913260	total: 96.1ms	remaining: 777ms
11:	learn: 0.3834286	total: 107ms	remaining: 785ms
12:	learn: 0.3782162	total: 115ms	remaining: 772ms
13:	learn: 0.3731039	total: 124ms	remaining: 763ms
14:	learn: 0.3661234	total: 134ms	remaining: 759ms
15:	learn: 0.3619679	total: 141ms	remaining: 740ms
16:	learn: 0.3580945	total: 150ms	remaining: 732ms
17:	learn: 0.3539507	total: 156ms	remaining: 711ms
18:	learn: 0.3507469	tot

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_nm, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_nm, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_nm, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_nm, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_nm, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_nm, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_nm, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_nm, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_nm, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_nm, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_nm, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_nm, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_nm, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_nm, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_nm, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_nm, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_nm, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_nm, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_nm, zero_division= 0))

Logistic Regression Precision: 0.11753048780487804
Decision Tree Precision: 0.12193152454780362
Tuned Decision Tree Precision: 0.15196599362380447
KNeighborsClassifier Precision: 0.1256011838697743
Tuned KNeighborsClassifier Precision: 0.18902038132807364
GaussianNB Precision: 0.11357113571135712
SVM Precision: 0.0990282276723739
Random Forest Precision: 0.12421085858585859
Tuned Random Forest Precision: 0.13761632068718682
AdaBoost Precision: 0.11707465409761289
Tuned AdaBoost Precision: 0.11786259541984732
GradientBoosting Precision: 0.13308436598200646
Tuned GradientBoosting Precision: 0.13308436598200646
XGB Precision: 0.12298450837812204
Tuned XGB Precision: 0.12646675358539766
LGBM Precision: 0.12397885631907736
Tuned LGBM Precision: 0.13959019392608854
CatBoost Precision: 0.12038745387453874
Tuned CatBoost Precision: 0.13443603730424072


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_nm, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_nm, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_nm, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_nm, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_nm, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_nm, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_nm, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_nm, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_nm, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_nm, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_nm, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_nm, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_nm, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_nm, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_nm, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_nm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_nm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_nm, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_nm, zero_division= 0)
                  }

nm_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
nm_precision= nm_precision.sort_values(by= 'Precision Score', ascending=False)
print(nm_precision)

                                    Model  Precision Score
4   Tuned KNeighborsClassifier Precision:         0.189020
2          Tuned Decision Tree Precision:         0.151966
16                  Tuned LGBM Precision:         0.139590
8          Tuned Random Forest Precision:         0.137616
18              Tuned CatBoost Precision:         0.134436
11            GradientBoosting Precision:         0.133084
12      Tuned GradientBoosting Precision:         0.133084
14                   Tuned XGB Precision:         0.126467
3         KNeighborsClassifier Precision:         0.125601
7                Random Forest Precision:         0.124211
15                        LGBM Precision:         0.123979
13                         XGB Precision:         0.122985
1                Decision Tree Precision:         0.121932
17                    CatBoost Precision:         0.120387
10              Tuned AdaBoost Precision:         0.117863
0          Logistic Regression Precision:         0.1175

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_nm),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_nm),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_nm),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_nm),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_nm),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_nm),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_nm),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_nm),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_nm),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_nm),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_nm),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_nm),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_nm),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_nm),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_nm),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_nm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_nm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_nm),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_nm)
                  }

nm_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
nm_accuracy= nm_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(nm_accuracy)

                                   Model  Accuracy Score
4   Tuned KNeighborsClassifier Accuracy:        0.702620
2          Tuned Decision Tree Accuracy:        0.555944
5                   GaussianNB Accuracy:        0.503447
16                  Tuned LGBM Accuracy:        0.485417
3         KNeighborsClassifier Accuracy:        0.473963
8          Tuned Random Forest Accuracy:        0.473751
18              Tuned CatBoost Accuracy:        0.462615
11            GradientBoosting Accuracy:        0.444798
12      Tuned GradientBoosting Accuracy:        0.444798
14                   Tuned XGB Accuracy:        0.417117
1                Decision Tree Accuracy:        0.406724
15                        LGBM Accuracy:        0.405345
7                Random Forest Accuracy:        0.398239
13                         XGB Accuracy:        0.397391
17                    CatBoost Accuracy:        0.379574
10              Tuned AdaBoost Accuracy:        0.372362
0          Logistic Regression 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_nm),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_nm),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_nm),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_nm),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_nm),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_nm),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_nm),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_nm),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_nm),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_nm),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_nm),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_nm),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_nm),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_nm),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_nm),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_nm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_nm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_nm),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_nm)
                  }

nm_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
nm_auc= nm_auc.sort_values(by= 'AUC Score', ascending=False)
print(nm_auc)

                              Model  AUC Score
4   Tuned KNeighborsClassifier AUC:   0.736839
16                  Tuned LGBM AUC:   0.733789
8          Tuned Random Forest AUC:   0.729173
18              Tuned CatBoost AUC:   0.723141
12      Tuned GradientBoosting AUC:   0.716175
11            GradientBoosting AUC:   0.716175
7                Random Forest AUC:   0.713528
14                   Tuned XGB AUC:   0.699105
15                        LGBM AUC:   0.696687
17                    CatBoost AUC:   0.689658
13                         XGB AUC:   0.689421
2          Tuned Decision Tree AUC:   0.686608
0          Logistic Regression AUC:   0.655698
3         KNeighborsClassifier AUC:   0.639709
1                Decision Tree AUC:   0.595196
6                          SVM AUC:   0.589588
10              Tuned AdaBoost AUC:   0.579354
9                     AdaBoost AUC:   0.577733
5                   GaussianNB AUC:   0.568494
